In [3]:
import os
from PIL import Image, ImageDraw, ImageFont, ImageOps
from moviepy.editor import VideoFileClip, ImageSequenceClip
import numpy as np

# Function to convert video to images
def video_to_images(video_path, output_dir, fps):
    clip = VideoFileClip(video_path)
    duration = clip.duration
    frame_rate = clip.fps
    frame_times = np.arange(0, duration, 1.0 / fps)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = []
    font_size = int(clip.size[1] * 0.05)    # 5% the height of the video
    font = ImageFont.truetype("arial.ttf", font_size)
    position = (10, 10)

    for timestamp in frame_times:
        img = clip.get_frame(timestamp)
        img_pil = Image.fromarray(img)
        timestamp_text = f"Time: {timestamp:.2f}s"

        # masking technique to draw inverted text on the image
        text_mask = Image.new("L", img_pil.size, 0)
        draw_mask = ImageDraw.Draw(text_mask)
        draw_mask.text(position, timestamp_text, fill=255, font=font)
        inverted_image = ImageOps.invert(img_pil)                           # invert the colors in the region of the text
        final_image = Image.composite(inverted_image, img_pil, text_mask)   # combine the original image with the inverted region using the text mask

        # save the resultant image to path (may or may not replace with in-memory-image-data-management)
        image_path = os.path.join(output_dir, f"f_{int(timestamp * fps):04d}.jpg")
        final_image.save(image_path, "JPEG")
        images.append(image_path)

    return images, frame_rate, fps

input_video_path = "input.mp4"
VideoFileClip("30_second_animation_assignment.mp4").write_videofile(input_video_path, codec='libx264', logger=None)
output_dir = "output"
print("Converting video to image array...")
images, original_fps, reduced_fps = video_to_images(input_video_path, output_dir, fps=2)
clip = ImageSequenceClip(images,fps=1)
print("Image array successfully created.")

Converting video to image array...
Image array successfully created.


In [4]:
from IPython.display import display, Video

# sanity check: ensure the image array has been edited correctly
output_video_path = "output.mp4"
clip.write_videofile(output_video_path, codec='libx264', logger=None)
display(Video(output_video_path, width=500))